<a href="https://colab.research.google.com/github/mukeshyadavm/cpra-legalbert-sequential-transfer/blob/main/LegalBERT_SNLI_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [ ]:
!pip uninstall -y datasets fsspec
!pip install --no-cache-dir git+https://github.com/huggingface/datasets.git
!pip install -q transformers evaluate


Found existing installation: datasets 4.0.1.dev0
Uninstalling datasets-4.0.1.dev0:
  Successfully uninstalled datasets-4.0.1.dev0
Found existing installation: fsspec 2025.3.0
Uninstalling fsspec-2025.3.0:
  Successfully uninstalled fsspec-2025.3.0
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-ojgu66jt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-ojgu66jt
  Resolved https://github.com/huggingface/datasets.git to commit 9dfa28814c8a34bea20dbc31e4f4376cf1a8c89e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.9 MB/s eta 0:00:00
  Created wheel for datasets: filename=datasets-4.0.1.dev0-py3-none-any.whl size=495229 sha256=690e68b058eea04aa55e33819cb1ba4e661d53663a331cfeb6b2d878f7202b96
  Stored in directory: /tmp/pip-ephem-wheel-cache-w6x4007w/

In [ ]:
!pip install transformers

In [ ]:
import transformers
print(transformers.__version__)

4.53.1


In [ ]:

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from sklearn.metrics import accuracy_score

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


In [ ]:
from datasets import load_dataset
dataset = load_dataset("snli")
dataset = {split: ds.filter(lambda x: x["label"] != -1) for split, ds in dataset.items()}

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

def tokenize_fn(example):
    return tokenizer(
        example["premise"],
        example["hypothesis"],
        truncation=True,
        padding="max_length"
    )

In [ ]:

tokenized_dataset = {
    split: ds.map(tokenize_fn, batched=True)
    for split, ds in dataset.items()
}


Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

Map:   0%|          | 0/549367 [00:00<?, ? examples/s]

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(
    "nlpaueb/legal-bert-base-uncased",
    num_labels=3
)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    fp16=True,
    dataloader_num_workers=4,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-15-1058956260.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
trainer.train()
trainer.save_model("/content/drive/MyDrive/legalbert-snli-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/legalbert-snli-finetuned")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.438500,0.340785,0.875127
2,0.340600,0.308915,0.884881
3,0.276200,0.329896,0.889555
4,0.222900,0.324885,0.889149
5,0.178700,0.383538,0.889250
6,0.142900,0.405817,0.890266
7,0.114200,0.472828,0.886710
8,0.089200,0.562102,0.887523
9,0.067400,0.609606,0.889047
10,0.050600,0.673343,0.888132


('/content/drive/MyDrive/legalbert-snli-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/legalbert-snli-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/legalbert-snli-finetuned/vocab.txt',
 '/content/drive/MyDrive/legalbert-snli-finetuned/added_tokens.json',
 '/content/drive/MyDrive/legalbert-snli-finetuned/tokenizer.json')

In [ ]:

model_path = "/content/drive/MyDrive/legalbert-snli-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    return {"accuracy": accuracy_score(pred.label_ids, preds)}

test_dataset = tokenized_dataset["test"].remove_columns(["premise", "hypothesis", "label"])
test_dataset = test_dataset.add_column("labels", dataset["test"]["label"])
test_dataset = test_dataset.with_format("torch")

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=test_dataset)
print(f" Test Accuracy: {metrics['eval_accuracy']:.4f}")

/tmp/ipython-input-59-4286577022.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


 Test Accuracy: 0.8849
